In [1]:
import json, boto3
from pprint import pprint

ec2_assume_role_policy_doc = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "ec2.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
})

print(ec2_assume_role_policy_doc)

{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "ec2.amazonaws.com"}, "Action": "sts:AssumeRole"}]}


In [3]:

gerald = boto3.Session(
    profile_name= 'gerald.njoku',
    region_name= "eu-central-1"
)

# add a a new client
gerald_IAM = gerald.client("iam")

# we create the role with the new client
response = gerald_IAM.create_role(
    RoleName = "ec2-readonly-gerald-2",
    AssumeRolePolicyDocument = ec2_assume_role_policy_doc,
    Description = "Allows an instance to ec2 read-only access",
    Tags = [
        {
            "Key": "gerald.njoku",
            "Value": "For automation with python"
        }
    ]
)

pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '941',
                                      'content-type': 'text/xml',
                                      'date': 'Mon, 05 Sep 2022 12:21:37 GMT',
                                      'x-amzn-requestid': '4a95f1f1-504a-438e-9a14-1c28d36fc6af'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4a95f1f1-504a-438e-9a14-1c28d36fc6af',
                      'RetryAttempts': 0},
 'Role': {'Arn': 'arn:aws:iam::318867684519:role/ec2-readonly-gerald-2',
          'AssumeRolePolicyDocument': {'Statement': [{'Action': 'sts:AssumeRole',
                                                      'Effect': 'Allow',
                                                      'Principal': {'Service': 'ec2.amazonaws.com'}}],
                                       'Version': '2012-10-17'},
          'CreateDate': datetime.datetime(2022, 9, 5, 12, 21, 37, tzinfo=tzutc()),
          'Path': '/',
          'RoleId': 'AROAUUP

In [4]:
ec2_readonly_role_name = response['Role']['RoleName']
print(ec2_readonly_role_name)




ec2-readonly-gerald-2


In [5]:
# we create the corresponding instance profile

response = gerald_IAM.create_instance_profile(
    InstanceProfileName   = ec2_readonly_role_name,
    Tags = [
        {
            "Key": "gerald.njoku",
            "Value": "For automation with python"
        },
    ]
)
pprint(response)

{'InstanceProfile': {'Arn': 'arn:aws:iam::318867684519:instance-profile/ec2-readonly-gerald-2',
                     'CreateDate': datetime.datetime(2022, 9, 5, 12, 24, 20, tzinfo=tzutc()),
                     'InstanceProfileId': 'AIPAUUPP6XSTRSSQLOSZX',
                     'InstanceProfileName': 'ec2-readonly-gerald-2',
                     'Path': '/',
                     'Roles': [],
                     'Tags': [{'Key': 'gerald.njoku',
                               'Value': 'For automation with python'}]},
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '788',
                                      'content-type': 'text/xml',
                                      'date': 'Mon, 05 Sep 2022 12:24:20 GMT',
                                      'x-amzn-requestid': '5ac91218-e18b-49ae-89bf-e26160dc93fa'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5ac91218-e18b-49ae-89bf-e26160dc93fa',
                      'RetryAttempts': 0}}


In [6]:
# we assign the EC2 Read Only managed policy to the new role

gerald_IAM.attach_role_policy(
    PolicyArn= 'arn:aws:iam::aws:policy/AmazonEC2ReadOnlyAccess',
    RoleName = ec2_readonly_role_name
)


# go to your AWS console and check the role again(ec2role we created)

{'ResponseMetadata': {'RequestId': 'd614a382-36e1-4522-88dd-d05d2ccde558',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd614a382-36e1-4522-88dd-d05d2ccde558',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 05 Sep 2022 12:25:53 GMT'},
  'RetryAttempts': 0}}

In [7]:
response = gerald_IAM.add_role_to_instance_profile(
    InstanceProfileName = ec2_readonly_role_name,
    RoleName = ec2_readonly_role_name
)

print(response)

{'ResponseMetadata': {'RequestId': '2691728e-9925-4f3a-b3b1-ab9957d70d2a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2691728e-9925-4f3a-b3b1-ab9957d70d2a', 'content-type': 'text/xml', 'content-length': '228', 'date': 'Mon, 05 Sep 2022 13:06:21 GMT'}, 'RetryAttempts': 0}}


In [8]:
# New client for EC2
gerald_EC2 = gerald.client("ec2")

# we attach the role for an instance
response = gerald_EC2.associate_iam_instance_profile(
    IamInstanceProfile = {
        "Name": ec2_readonly_role_name
    },
    # remember to put it in the same region, if not, errors go choke  
    InstanceId  = "i-0e6ab39d2eefef734"
)

print(response)

ClientError: An error occurred (IncorrectState) when calling the AssociateIamInstanceProfile operation: There is an existing association for instance i-0e6ab39d2eefef734

we create a fucntion that does everything so we don't need to remind all of the steps, automate the process, and make sure of the rules are followed

In [8]:
from time import sleep

def grant_role_to_instance (session: boto3.session.Session, instance_id: str,
                            role_name: str, role_description:str,
                            policy_to_grant: str) -> None:
    
    # this policy allows only EC2 instances to assume this role
    ec2_assume_role_policy_doc = json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "ec2.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    })

    # IAM Client
    iam  =  session.client("iam")

    # we create the role with the new client
    iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = ec2_assume_role_policy_doc,
        Description = role_description
    )

    # we create the corresponding instance profile
    iam.create_instance_profile(
        InstanceProfileName = role_name,
    )

    # we assign the ec2 Read Only managed policy to the new role
    iam.attach_role_policy(
        PolicyArn=policy_to_grant,
        RoleName = role_name

    )

    # we assign the role to the instance profile
    iam.add_role_to_instance_profile(
        InstanceProfileName= role_name,
        RoleName = role_name
    )

    # wait for the changes to propagate
    sleep(10)
    # EC2 client
    ec2 = session.client("ec2")
    # we attach the role to the instance
    ec2.associate_iam_instance_profile(
        IamInstanceProfile={
            "Name": role_name
        },
        InstanceId = instance_id
    )


In [10]:
grant_role_to_instance(session = gerald, instance_id="i-08ea3dc227cf38ff4", role_name= "fucntion_role7",
                        role_description= "created via another function", 
                        policy_to_grant="arn:aws:iam::aws:policy/AmazonEC2ReadOnlyAccess")
                        

# AWS IS ASYNCHRONOUS
Many times the calls to the AWS API return with a "success" code, even if the requested action hasn't finished yet.
For example, the function fails because the instance profile is not ready when we try to assign it to the instance


# Waiters
There are specific commands to **"wait"** for a resource to be ready

example:

    ```python
    waiter = iam.get_waiter("instance_profile_exists)
    waiter.wait(
        InstanceProfileName = 'string',
        WaiterConfig={
            "Delay": 123, 
            "MaxAttempts: 123
        }
    )
    ```

In [7]:
# GET INFORMATION IF THERE ARE NOT MUCH DATA


ec2 = gerald.client("ec2")
response = ec2.describe_instances()
pprint(response, depth=1)

{'Reservations': [...], 'ResponseMetadata': {...}}


In [8]:
print(len(response["Reservations"]))

3


In [9]:
pprint(response["Reservations"][0], depth=1)


{'Groups': [],
 'Instances': [...],
 'OwnerId': '318867684519',
 'ReservationId': 'r-0a3f7d6cbbb7b850d'}


In [11]:
print(len(response["Reservations"][0]["Instances"]))


1


In [12]:
pprint(response["Reservations"][0]["Instances"][0], depth=1)

{'AmiLaunchIndex': 0,
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [...],
 'CapacityReservationSpecification': {...},
 'ClientToken': '',
 'CpuOptions': {...},
 'EbsOptimized': False,
 'EnaSupport': True,
 'EnclaveOptions': {...},
 'HibernationOptions': {...},
 'Hypervisor': 'xen',
 'IamInstanceProfile': {...},
 'ImageId': 'ami-0e2031728ef69a466',
 'InstanceId': 'i-06bd4f1e00234edd2',
 'InstanceType': 't2.micro',
 'Ipv6Address': '2a05:d014:e31:c200:a1b0:2867:fc9f:1629',
 'KeyName': 'soso',
 'LaunchTime': datetime.datetime(2022, 8, 30, 16, 56, 59, tzinfo=tzutc()),
 'MaintenanceOptions': {...},
 'MetadataOptions': {...},
 'Monitoring': {...},
 'NetworkInterfaces': [...],
 'Placement': {...},
 'PlatformDetails': 'Linux/UNIX',
 'PrivateDnsName': 'ip-10-0-6-64.eu-central-1.compute.internal',
 'PrivateDnsNameOptions': {...},
 'PrivateIpAddress': '10.0.6.64',
 'ProductCodes': [],
 'PublicDnsName': 'ec2-18-196-82-51.eu-central-1.compute.amazonaws.com',
 'PublicIpAddress': '18.196.82.51',

In [23]:
print(len(response["Reservations"][0]["Instances"][0]["Tags"]))
pprint(response["Reservations"][0]["Instances"][0]["Tags"][0])


1
{'Key': 'Name', 'Value': 'test'}


In [24]:
# to get every running EC2 Instances
for reservation in response["Reservations"]:
    for instance in reservation["Instances"]:
        print(f"{instance['InstanceId']} is {[tag['Value'] for tag in instance['Tags'] ] [0]}")

i-06bd4f1e00234edd2 is test
i-0794bdae0f118bae7 is another_one
i-0673f5415cc1cd616 is test-authomate2


## Filtering Our Queries

In [26]:
response = ec2.describe_instances(InstanceIds=["i-0794bdae0f118bae7"])
pprint(response["Reservations"][0]["Instances"][0])

{'AmiLaunchIndex': 0,
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
                          'Ebs': {'AttachTime': datetime.datetime(2022, 8, 30, 17, 48, 21, tzinfo=tzutc()),
                                  'DeleteOnTermination': True,
                                  'Status': 'attached',
                                  'VolumeId': 'vol-0329c6f03cbae6cc1'}}],
 'CapacityReservationSpecification': {'CapacityReservationPreference': 'open'},
 'ClientToken': '',
 'CpuOptions': {'CoreCount': 1, 'ThreadsPerCore': 1},
 'EbsOptimized': False,
 'EnaSupport': True,
 'EnclaveOptions': {'Enabled': False},
 'HibernationOptions': {'Configured': False},
 'Hypervisor': 'xen',
 'IamInstanceProfile': {'Arn': 'arn:aws:iam::318867684519:instance-profile/fucntion_role2',
                        'Id': 'AIPAUUPP6XSTRVXNKWP5Z'},
 'ImageId': 'ami-0e2031728ef69a466',
 'InstanceId': 'i-0794bdae0f118bae7',
 'InstanceType': 't2.micro',
 'KeyName': 'soso',
 'LaunchTime': date

In [35]:
# check for instnaces which security group is = bastion
response  = ec2.describe_instances(
    Filters=[
        {
            "Name": "instance.group-name",
            "Values": ["bastion"]
        }
    ]
)

pprint(response["Reservations"])

[]


In [36]:
# getting tags by color
response  = ec2.describe_instances(
    Filters=[
        {
            "Name": "tag:color",
            "Values": ["blue"]
        }
    ]
)

pprint(response["Reservations"][0]["Instances"][0])

IndexError: list index out of range

# if there is too much data: Pagination

In [61]:
response = geraldIAM.list_policies(
    Scope = "All",
    OnlyAttached= False,
    PolicyUsageFilter="PermissionsPolicy",
    MaxItems=10
)
pprint(response, depth=1)
print("---------")
for policy in response["Policies"]:
    print("  "+ policy["PolicyName"])
next_marker = response["Marker"]

{'IsTruncated': True,
 'Marker': 'AApBd43+fZdHWd2URH3bUHkqQQ1pnjqod0NaHOsPnAD8gkv5Rj2fJtYlUg7fEZRA4EB0TGa3zUkiIPc2rxNeSxrlT+JgztIbsKcS0vp0ZHfR/g==',
 'Policies': [...],
 'ResponseMetadata': {...}}
---------
  AWSLambdaBasicExecutionRole-17789289-ddbf-4a03-8258-4703b1820b7b
  AWSLambdaBasicExecutionRole-278bf050-1567-44d0-9dc1-11715bb39f7e
  AWSLambdaBasicExecutionRole-286202bc-27f8-438f-b562-59da099152b5
  AWSLambdaBasicExecutionRole-41de3edb-cf18-41de-936a-d16d109cfc35
  AWSLambdaBasicExecutionRole-47d96e6c-2a8f-4397-b332-9b7f27d436f7
  AWSLambdaBasicExecutionRole-5d76cc2e-d983-45d2-8443-bb0e925479a1
  AWSLambdaBasicExecutionRole-7e5f1432-fef3-4204-ba90-cccaa5f5705b
  AWSLambdaBasicExecutionRole-80b62a12-efe2-4ece-81aa-5588f405dc81
  AWSLambdaBasicExecutionRole-822eef52-6dd0-4506-9d57-1092920deb73
  AWSLambdaBasicExecutionRole-9ffdb872-ec09-47cf-836b-7d871f7da4e2


In [68]:
response = geraldIAM.list_policies(
    Scope="All",
    OnlyAttached=False,
    PolicyUsageFilter="PermissionsPolicy",
    MaxItems =10,
    Marker=next_marker
) 

pprint(response, depth=1)
print("---------------------------")
for policy in response["Policies"]:
    print("  " + policy["PolicyName"])
next_marker = response["Marker"]

{'IsTruncated': True,
 'Marker': 'ABsjhN2J+YtrrodFXP01m0nhQyM2qtDb0OE5MAxlx2O7dPA1TOgk+mwXiNuw44nRPBweO0azhHDH2nRkxVn7hTF4zxnjCtAbTwsb1JeSaVpD2g==',
 'Policies': [...],
 'ResponseMetadata': {...}}
---------------------------
  AWSLambdaBasicExecutionRole-a3009e52-5e32-4e34-ac14-b7afa663ceda
  AWSLambdaBasicExecutionRole-aa32ba0e-0c89-4c05-8115-a14e8c408d18
  AWSLambdaBasicExecutionRole-ac27b1cf-59bc-4333-9586-488042cbe01c
  AWSLambdaBasicExecutionRole-c66acd91-ad2e-40a0-ba12-9dc85174e81c
  AWSLambdaBasicExecutionRole-d475eb2e-f734-45c7-9da7-1008bd5b9c68
  AWSLambdaBasicExecutionRole-da7599c0-d1c7-4d4e-a99e-5a68e4f89ad6
  AWSLambdaBasicExecutionRole-f9e39ee4-dfa9-4219-87bf-9400c6d6117b
  iam_policy_1
  kinesis-analytics-service-wildrydes-eu-west-2
  ListBucket
